In [18]:
# URL for the single day's data
url_single_day = "https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=20250704&type=ALL&response=csv"
date_single_day_str = "20250704"

print(f"Processing data for {date_single_day_str}...")
try:
    raw_text_single_day = download_twse_csv(date_single_day_str)
    lines_single_day = extract_csv_lines(raw_text_single_day)
    header_line_single_day, data_lines_single_day = find_data_section(lines_single_day)
    df_single_day = clean_data(header_line_single_day, data_lines_single_day)
    df_single_day['日期'] = pd.to_datetime(date_single_day_str).strftime('%Y-%m-%d') # Add date column

    print(f"✅ Successfully processed data for {date_single_day_str}")

    # Display the DataFrame information
    print("\nSingle Day DataFrame Info:")
    df_single_day.info()

    # Display the head of the DataFrame
    print("\nSingle Day DataFrame Head:")
    display(df_single_day.head())

except Exception as e:
    print(f"❌ Failed to process data for {date_single_day_str}: {e}")

Processing data for 20250704...
✅ Download complete for 20250704
✅ Extracted 32160 non-empty lines
✅ Found header at line 306
Skipping row due to incorrect column count: ['漲跌(+/-)欄位符號說明:+/-/X表示漲/跌/不比價。', '']
Skipping row due to incorrect column count: ['當證券代號為認購(售)權證及認股權憑證時本益比欄位置為結算價；但如為以外國證券或指數為標的之認購(售)權證及履約方式採歐式者，該欄位為空白。', '']
Skipping row due to incorrect column count: ['除境外指數股票型基金及外國股票第二上市外，餘交易單位皆為千股。', '']
Skipping row due to incorrect column count: ['本統計資訊含一般、零股、盤後定價、鉅額交易，不含拍賣、標購。', '']
✅ Successfully processed data for 20250704

Single Day DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31849 entries, 0 to 31848
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   證券代號     31849 non-null  object 
 1   證券名稱     31849 non-null  object 
 2   成交股數     18725 non-null  float64
 3   成交筆數     31478 non-null  float64
 4   成交金額     19122 non-null  float64
 5   開盤價      13113 non-null  float64
 6   最高價      1311

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,,日期
0,0050,元大台灣50,NaN,NaN,NaN,49.40,49.49,48.83,48.86,-,0.46,48.86,601.0,48.87,153.0,0.0,,2025-07-04
1,0051,元大中型100,NaN,229.0,NaN,76.10,76.65,75.60,75.60,-,0.45,75.50,1.0,75.65,2.0,0.0,,2025-07-04
2,0052,富邦科技,NaN,952.0,NaN,191.85,192.00,189.50,190.10,-,1.35,190.10,2.0,190.20,5.0,0.0,,2025-07-04
3,0053,元大電子,NaN,123.0,NaN,105.00,105.15,104.10,104.10,-,0.95,103.75,1.0,104.05,1.0,0.0,,2025-07-04
4,0055,元大MSCI金融,NaN,225.0,NaN,29.23,29.26,29.01,29.06,-,0.17,29.03,1.0,29.11,1.0,0.0,,2025-07-04


In [23]:
if 'all_data' in locals() and not all_data.empty:
    print("Dates in all_data DataFrame:")
    display(all_data['日期'].unique())
else:
    print("all_data DataFrame is empty or not found.")

Dates in all_data DataFrame:


array(['2025-06-30', '2025-07-01', '2025-07-02', '2025-07-03',
       '2025-07-04'], dtype=object)

In [20]:
# Group by stock code and name, and aggregate key metrics
# Using .agg() allows applying multiple aggregation functions to different columns
aggregated_data = all_data.groupby(['證券代號', '證券名稱']).agg(
    總成交股數=('成交股數', 'sum'),
    總成交筆數=('成交筆數', 'sum'),
    總成交金額=('成交金額', 'sum'),
    平均開盤價=('開盤價', 'mean'),
    最高價=('最高價', 'max'),
    最低價=('最低價', 'min'),
    平均收盤價=('收盤價', 'mean'),
    平均漲跌價差=('漲跌價差', 'mean'),
    總最後揭示買量=('最後揭示買量', 'sum'),
    總最後揭示賣量=('最後揭示賣量', 'sum'),
    平均本益比=('本益比', 'mean'),
    交易天數=('日期', 'nunique') # Count unique dates for each stock
).reset_index() # Reset index to turn grouped columns back into regular columns

# Display the aggregated data
print("\nAggregated Stock Data:")
display(aggregated_data.head())

# Display info of the aggregated DataFrame
print("\nAggregated Stock Data Info:")
aggregated_data.info()


Aggregated Stock Data:


,證券代號,證券名稱,總成交股數,總成交筆數,總成交金額,平均開盤價,最高價,最低價,平均收盤價,平均漲跌價差,總最後揭示買量,總最後揭示賣量,平均本益比,交易天數
0,0050,元大台灣50,0.0,0.0,0.0,48.814,49.60,48.24,48.766,0.298,1002.0,619.0,0.0,5
1,0051,元大中型100,0.0,1362.0,0.0,75.410,76.65,74.60,75.330,0.340,63.0,23.0,0.0,5
2,0052,富邦科技,0.0,1604.0,0.0,189.750,192.30,187.45,189.660,1.000,24.0,30.0,0.0,5
3,0053,元大電子,0.0,642.0,0.0,103.780,105.15,102.40,103.770,0.870,60.0,56.0,0.0,5
4,0055,元大MSCI金融,0.0,1321.0,0.0,29.096,29.29,28.82,29.140,0.150,58.0,24.0,0.0,5



Aggregated Stock Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32760 entries, 0 to 32759
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   證券代號     32760 non-null  object 
 1   證券名稱     32760 non-null  object 
 2   總成交股數    32760 non-null  float64
 3   總成交筆數    32760 non-null  float64
 4   總成交金額    32760 non-null  float64
 5   平均開盤價    19941 non-null  float64
 6   最高價      19941 non-null  float64
 7   最低價      19941 non-null  float64
 8   平均收盤價    19941 non-null  float64
 9   平均漲跌價差   32760 non-null  float64
 10  總最後揭示買量  32760 non-null  float64
 11  總最後揭示賣量  32760 non-null  float64
 12  平均本益比    28775 non-null  float64
 13  交易天數     32760 non-null  int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 3.5+ MB


In [21]:
# Sort the aggregated data by total trade amount in descending order
highest_trade_amount_stock = aggregated_data.sort_values(by='總成交金額', ascending=False).head(1)

# Display the stock with the highest total trade amount
print("\nStock with the highest total trade amount (2025-06-30 to 2025-07-04):")
display(highest_trade_amount_stock)


Stock with the highest total trade amount (2025-06-30 to 2025-07-04):


,證券代號,證券名稱,總成交股數,總成交筆數,總成交金額,平均開盤價,最高價,最低價,平均收盤價,平均漲跌價差,總最後揭示買量,總最後揭示賣量,平均本益比,交易天數
4185,055403,一詮統一53購01,0.0,5.0,4000.0,0.08,0.08,0.08,0.08,0.002,90.0,0.0,76.82,5


In [22]:
# Find the stock with the highest value for selected columns

# Stock with the highest total trade volume
highest_volume_stock = aggregated_data.sort_values(by='總成交股數', ascending=False).head(1)
print("\nStock with the highest total trade volume:")
display(highest_volume_stock)

# Stock with the highest total trade amount (already done, but repeating for context)
highest_amount_stock = aggregated_data.sort_values(by='總成交金額', ascending=False).head(1)
print("\nStock with the highest total trade amount:")
display(highest_amount_stock)


# Stock with the highest average closing price
highest_avg_close_stock = aggregated_data.sort_values(by='平均收盤價', ascending=False).head(1)
print("\nStock with the highest average closing price:")
display(highest_avg_close_stock)


Stock with the highest total trade volume:


,證券代號,證券名稱,總成交股數,總成交筆數,總成交金額,平均開盤價,最高價,最低價,平均收盤價,平均漲跌價差,總最後揭示買量,總最後揭示賣量,平均本益比,交易天數
21,00636K,國泰中國A50+U,1500.0,13.0,0.0,7.35,7.47,7.32,7.37,0.036,10.0,22.0,0.0,5



Stock with the highest total trade amount:


,證券代號,證券名稱,總成交股數,總成交筆數,總成交金額,平均開盤價,最高價,最低價,平均收盤價,平均漲跌價差,總最後揭示買量,總最後揭示賣量,平均本益比,交易天數
4185,055403,一詮統一53購01,0.0,5.0,4000.0,0.08,0.08,0.08,0.08,0.002,90.0,0.0,76.82,5



Stock with the highest average closing price:


,證券代號,證券名稱,總成交股數,總成交筆數,總成交金額,平均開盤價,最高價,最低價,平均收盤價,平均漲跌價差,總最後揭示買量,總最後揭示賣量,平均本益比,交易天數
31997,2383,台光電,0.0,0.0,0.0,900.6,963.0,869.0,907.2,13.4,144.0,102.0,28.422,5
